In [1]:
from nbdev.showdoc import show_doc

In [2]:
from nbdev.imports import *
from fastcore.script import *
from fastcore.foundation import *
from keyword import iskeyword
import nbformat

# Export to scala sbt project 

> The functions that transform notebooks in a library (nearly 1:1 translation from nbdev 00_export.ipynb, with the goal of transforming notebooks using Jupyter Scala kernel (https://almond.sh) into importable, testable, and well documented Scala files. 

The most important function defined in this module is `notebooks2script`, so you may want to jump to it before scrolling though the rest, which explain the details behind the scenes of the conversion from notebooks to library. The main things to remember are:
- put `# export` on each cell you want exported
- put `# exports` on each cell you want exported with the source code shown in the docs 
- put `# exporti` on each cell you want exported without it being added to `__all__`, and without it showing up in the docs.
- one cell should contain `# default_exp` followed by the name of the module (with points for submodules and without the py extension) everything should be exported in (if one specific cell needs to be exported in a different module, just indicate it after `#export`: `#export special.module`)
- all left members of an equality, functions and classes will be exported and variables that are not private will be put in the `__all__` automatically
- to add something to `__all__` if it's not picked automatically, write an exported cell with something like `#add2all "my_name"`

## Basic foundations

For bootstrapping `nbdev` we have a few basic foundations defined in <code>imports</code>, which we test a show here. First, a simple config file class, `Config` that read the content of your `settings.ini` file and make it accessible:

In [3]:
show_doc(Config, title_level=3)

<h3 id="Config" class="doc_header"><code>Config</code><a href="https://github.com/fastai/fastcore/tree/master/fastcore/foundation.py#L252" class="source_link" style="float:right">[source]</a></h3>

> <code>Config</code>(**`cfg_name`**=*`'settings.ini'`*)

Reading and writing `settings.ini`

In [4]:
create_config("github", "chisel-nbdev", user='jason', path='..', tst_flags='tst', cfg_name='test_settings.ini')
cfg = Config(cfg_name='settings.ini')
test_eq(cfg.lib_name, 'chisel-nbdev')
test_eq(cfg.git_url, "https://github.com/ucsc-vama/chisel-nbdev")
# test_eq(cfg.path("lib_path"), Path.cwd().parent/'nbdev')
# test_eq(cfg.path("nbs_path"), Path.cwd())
# test_eq(cfg.path("doc_path"), Path.cwd().parent/'docs')
test_eq(cfg.custom_sidebar, 'False')

## Reading a notebook

### What's a notebook?

A jupyter notebook is a json file behind the scenes. We can just read it with the json module, which will return a nested dictionary of dictionaries/lists of dictionaries, but there are some small differences between reading the json and using the tools from `nbformat` so we'll use this one.

In [5]:
#export
def read_nb(fname):
    "Read the notebook in `fname`."
    with open(Path(fname),'r', encoding='utf8') as f: return nbformat.reads(f.read(), as_version=4)

`fname` can be a string or a pathlib object.

In [6]:
test_nb = read_nb('test.ipynb')

The root has four keys: `cells` contains the cells of the notebook, `metadata` some stuff around the version of python used to execute the notebook, `nbformat` and `nbformat_minor` the version of nbformat. 

In [7]:
test_nb.keys()

dict_keys(['cells', 'metadata', 'nbformat', 'nbformat_minor'])

In [8]:
test_nb['metadata']

{'kernelspec': {'display_name': 'Scala', 'language': 'scala', 'name': 'scala'},
 'language_info': {'codemirror_mode': 'text/x-scala',
  'file_extension': '.scala',
  'mimetype': 'text/x-scala',
  'name': 'scala',
  'nbconvert_exporter': 'script',
  'version': '2.12.10'}}

In [9]:
f"{test_nb['nbformat']}.{test_nb['nbformat_minor']}"

'4.4'

The cells key then contains a list of cells. Each one is a new dictionary that contains entries like the type (code or markdown), the source (what is written in the cell) and the output (for code cells).

In [10]:
test_nb['cells'][0]

{'cell_type': 'markdown',
 'metadata': {},
 'source': '### Call the load_ivy script.\nLocated in the source folder, the load_ivy.sc script will use ammonite to download snapshots of the chisel libraries, to be cached on your system. This is necessary for EVERY chisel notebook so we can then import our chisel libraries like we would in an SBT project structure.'}

### Finding patterns

The following functions are used to catch the flags used in the code cells.

In [11]:
#export
def check_re(cell, pat, code_only=True):
    "Check if `cell` contains a line with regex `pat`"
    if code_only and cell['cell_type'] != 'code': return
    if isinstance(pat, str): pat = re.compile(pat, re.IGNORECASE | re.MULTILINE)
    return pat.search(cell['source'])

`pat` can be a string or a compiled regex. If `code_only=True`, this function ignores non-code cells, such as markdown.

In [12]:
cell = test_nb['cells'][1].copy()
# print(cell)
assert check_re(cell, '//export') is not None
assert check_re(cell, re.compile('//export')) is not None
assert check_re(cell, '# bla') is None
cell['cell_type'] = 'markdown'
assert check_re(cell, '//export') is None # don't export markdown
assert check_re(cell, '//export', code_only=False) is not None # unless specified

In [13]:
#export
def check_re_multi(cell, pats, code_only=True):
    "Check if `cell` contains a line matching any regex in `pats`, returning the first match found"
    return L(pats).map_first(partial(check_re, cell, code_only=code_only))

In [14]:
cell = test_nb['cells'][1].copy()
cell['source'] = "a b c"
# print(cell)
assert check_re(cell, 'a') is not None
assert check_re(cell, 'd') is None
# show that searching with patterns ['d','b','a'] will match 'b'
# i.e. 'd' is not found and we don't search for 'a'
assert check_re_multi(cell, ['d','b','a']).span() == (2,3)

In [15]:
#export
def _mk_flag_re(body, n_params, comment):
    "Compiles a regex for finding nbdev flags"
    assert body!=True, 'magics no longer supported'
    prefix = r"\s*\//\s*" # scala style comment "//" is prefix
    param_group = ""
    if n_params == -1: param_group = r"[ \t]+(.+)"
    if n_params == 1: param_group = r"[ \t]+(\S+)"
    if n_params == (0,1): param_group = r"(?:[ \t]+(\S+))?"
    return re.compile(rf"""
# {comment}:
^           # beginning of line (since re.MULTILINE is passed)
{prefix}
{body}
{param_group}
[ \t]*       # any number of spaces and/or tabs
$            # end of line (since re.MULTILINE is passed)
""", re.MULTILINE | re.VERBOSE)

This function returns a regex object that can be used to find nbdev flags in multiline text
- `body` regex fragment to match one or more flags,
- `n_params` number of flag parameters to match and catch (-1 for any number of params; `(0,1)` for 0 for 1 params),
- `comment` explains what the compiled regex should do.

In [16]:
#hide
re_blank_test = _mk_flag_re('export[si]?', 0, "test") # 0:
re_mod_test = _mk_flag_re('export[si]?', 1, "test")
re_opt_test = _mk_flag_re('export[si]?', (0,1), "test")
for f in ['export', 'exports', 'exporti']:
    cell = nbformat.v4.new_code_cell(f'//{f}  \n some code')
    assert check_re(cell, re_blank_test) is not None
    assert check_re(cell, re_mod_test) is None
    assert check_re(cell, re_opt_test) is not None
    test_eq(check_re(cell, re_opt_test).groups()[0], None)
    cell.source = f'//{f} special.module \n some code'
    assert check_re(cell, re_blank_test) is None
    assert check_re(cell, re_mod_test) is not None
    test_eq(check_re(cell, re_mod_test).groups()[0], 'special.module')
    assert check_re(cell, re_opt_test) is not None
    test_eq(check_re(cell, re_opt_test).groups()[0], 'special.module')

In [17]:
#export
_re_blank_export = _mk_flag_re("export[si]?", 0,
    "Matches any line with //export, //exports or //exporti without any module name")

In [18]:
#export
_re_mod_export = _mk_flag_re("export[si]?", 1,
    "Matches any line with //export, //exports or //exporti with a module name and catches it in group 1")

In [19]:
#export
_re_internal_export = _mk_flag_re("exporti", (0,1),
    "Matches any line with //exporti with or without a module name")

In [20]:
#exporti
def _is_external_export(tst):
    "Check if a cell is an external or internal export. `tst` is an re match"
    return _re_internal_export.search(tst.string) is None

In [21]:
#export
def is_export(cell, default):
    "Check if `cell` is to be exported and returns the name of the module to export it if provided"
    tst = check_re(cell, _re_blank_export)
    if tst:
        if default is None:
            print(f"No export destination, ignored:\n{cell['source']}")
        return default, _is_external_export(tst)
    tst = check_re(cell, _re_mod_export)
    if tst: return os.path.sep.join(tst.groups()[0].split('.')), _is_external_export(tst)
    else: return None

`is_export` returns;
- a tuple of ("module name", "external boolean" (`False` for an internal export)) if `cell` is to be exported or 
- `None` if `cell` will not be exported.

The cells to export are marked with `//export`/`//exporti`/`//exports`, potentially with a module name where we want it exported. The default module is given in a cell of the form `//default_exp bla` inside the notebook (usually at the top), though in this function, it needs the be passed (the final script will read the whole notebook to find it).
- a cell marked with `//export`/`//exporti`/`//exports` will be exported to the default module
- an exported cell marked with `special.module` appended will be exported in `special.module` (located in `lib_name/special/module.py`)
- a cell marked with `//export` will have its signature added to the documentation
- a cell marked with `//exports` will additionally have its source code added to the documentation
- a cell marked with `//exporti` will not show up in the documentation, and will also not be added to `__all__`.

In [22]:
cell = test_nb['cells'][1].copy()
test_eq(is_export(cell, 'export'), ('export', True))
cell['source'] = "// exports"
test_eq(is_export(cell, 'export'), ('export', True))
cell['source'] = "// exporti"
test_eq(is_export(cell, 'export'), ('export', False))
cell['source'] = "// export mod"
test_eq(is_export(cell, 'export'), ('mod', True))

In [23]:
#hide
cell['source'] = "// export mod.file"
test_eq(is_export(cell, 'export'), (f'mod{os.path.sep}file', True))
cell['source'] = "// exporti mod.file"
test_eq(is_export(cell, 'export'), (f'mod{os.path.sep}file', False))
cell['source'] = "// expt mod.file"
assert is_export(cell, 'export') is None
cell['source'] = "// exportmod.file"
assert is_export(cell, 'export') is None
cell['source'] = "// exportsmod.file"
assert is_export(cell, 'export') is None
cell['source'] = "// exporti mod file"
assert is_export(cell, 'export') is None

In [24]:
#export
_re_default_exp = _mk_flag_re('default_exp', 1, "Matches any line with //default_exp with a module name")

In [25]:
#export
def find_default_export(cells):
    "Find in `cells` the default export module."
    res = L(cells).map_first(check_re, pat=_re_default_exp)
    return res.groups()[0] if res else None

Stops at the first cell containing `// default_exp` (if there are several) and returns the value behind. Returns `None` if there are no cell with that code.

In [26]:
test_eq(find_default_export(test_nb['cells']), 'export_scala')
assert find_default_export(test_nb['cells'][2:]) is None

In [28]:
#hide
mods = [f'mod{i}' for i in range(3)]
cells = [{'cell_type': 'code', 'source': f'//default_exp {mod}'} for mod in mods]
for i, mod in enumerate(mods): test_eq(mod, find_default_export(cells[i:]))

### Listing all exported objects

Until now the above code has been verbatim to the ```00_export.ipynb``` from nbdev minus changes to regexes swapping '#' to '//'.
Now Scala syntax parsing is required as well as a target build infrastructure (sbt, mill, or Ammonite scripts).

The following functions make a list of everything that is exported to prepare a proper `__all__` for our exported module.

In [29]:
#export
_re_patch_func = re.compile(r"""
# Catches any function decorated with @patch, its name in group 1 and the patched class in group 2
@patch         # At any place in the cell, something that begins with @patch
(?:\s*@.*)*    # Any other decorator applied to the function
\s*def         # Any number of whitespace (including a new line probably) followed by def
\s+            # One whitespace or more
([^\(\s]+)     # Catch a group composed of anything but whitespace or an opening parenthesis (name of the function)
\s*\(          # Any number of whitespace followed by an opening parenthesis
[^:]*          # Any number of character different of : (the name of the first arg that is type-annotated)
:\s*           # A column followed by any number of whitespace
(?:            # Non-catching group with either
([^,\s\(\)]*)  #    a group composed of anything but a comma, a parenthesis or whitespace (name of the class)
|              #  or
(\([^\)]*\)))  #    a group composed of something between parenthesis (tuple of classes)
\s*            # Any number of whitespace
(?:,|\))       # Non-catching group with either a comma or a closing parenthesis
""", re.VERBOSE)